In [ ]:
import numpy as np
from gtoc13 import YPTU, SPTU, KMPDU, bodies_data
from gtoc13.path_finding.binlp.b_utils import min_dv_lam, plot_porkchop
from itertools import product
from IPython.display import clear_output
import time
from pathlib import Path

In [ ]:
# Timeframe
Yo = 3
Yf = 15
gridsize = 45
dv_lim = 30  # km/s
generate_all = False
show_plots = False
clear_plots = False
max_count = 5
K = 10
M = 9

In [ ]:
To = float(Yo / YPTU)
Tf = float(Yf / YPTU)
ts = np.logspace(To, Tf, gridsize, base=np.e)
dts = np.linspace((0 + (1 / 52)) / YPTU, (Yf - Yo) / YPTU, gridsize)
t_dep = np.log(ts)
tof = dts

In [ ]:
keys = [k for k, b in bodies_data.items() if b.is_planet() or b.name == "Yandi"]
KM_idxs = [(k, m) for (k, m) in product(keys, keys) if k != m]

In [ ]:
if generate_all:
    count = 0
    dvs = {}
    for km in KM_idxs:
        count += 1
        k, m = km
        min_dvs = [[min_dv_lam(k, ti, m, ti + dt) for dt in tof] for ti in t_dep]
        min_idx = np.unravel_index(np.argmin(min_dvs), (len(t_dep), len(tof)))
        dvs[km] = (
            np.round(min_dvs[min_idx[0]][min_idx[1]], 3),
            (
                np.round(t_dep[min_idx[0]] * float(YPTU), 3),
                np.round(tof[min_idx[1]] * float(YPTU), 3),
            ),
        )
        plot_porkchop(
            body1=k, body2=m, t_dep=t_dep, tof=tof, dvs=min_dvs, dv_limit=dv_lim, show=show_plots
        )
        if clear_plots and count < 5:
            time.sleep(5)
            clear_output(wait=True)
            if count == max_count:
                break
else:
    dvs = [[min_dv_lam(K, ti, M, dt) for dt in tof] for ti in t_dep]
    plot_porkchop(body1=K, body2=M, t_dep=t_dep, tof=tof, dvs=dvs, dv_limit=dv_lim, show=show_plots)
    print(np.round(np.min(dvs), 3), np.round(np.max(dvs), 3))
    print(t_dep)

In [ ]:
# min_dv_arcs = sorted([(k, v) for k, v in dvs.items()], key=lambda x: x[1][0])
# with open(Path.cwd() / "outputs/by_min_dv.txt", "w+") as f:
#     f.write("((ID1, ID2),(min_dv,(t_dep, tof)))")
#     for each in min_dv_arcs:
#         print(each)
#         f.write(str(each) + "\n")

In [ ]:
# min_dv_arcs = sorted([(k, v) for k, v in dvs.items()], key=lambda x: (x[1][1][0], x[1][0]))
# with open(Path.cwd() / "outputs/by_starting_t_and_dv.txt", "w+") as f:
#     f.write("((ID1, ID2),(min_dv,(t_dep, tof)))")
#     for each in min_dv_arcs:
#         print(each)
#         f.write(str(each) + "\n")


In [ ]:
# min_dv_arcs = sorted([(k, v) for k, v in dvs.items()], key=lambda x: (x[1][1][0], x[1][1][1]))
# with open(Path.cwd() / "outputs/by_starting_t_and_dt.txt", "w+") as f:
#     f.write("((ID1, ID2),(min_dv,(t_dep, tof)))")
#     for each in min_dv_arcs:
#         print(each)
#         f.write(str(each) + "\n")